<a href="https://colab.research.google.com/github/tcc3281/ATBMTT_code/blob/main/YOLOv8_Traffic_Sign_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
ahemateja19bec1025_trafficsignlocalizationdetectionyoloannotated_path = kagglehub.dataset_download('ahemateja19bec1025/trafficsignlocalizationdetectionyoloannotated')
keras_yolov8_keras_yolo_v8_s_backbone_coco_2_path = kagglehub.model_download('keras/yolov8/Keras/yolo_v8_s_backbone_coco/2')

print('Data source import complete.')


Data source import complete.


In [44]:
ahemateja19bec1025_trafficsignlocalizationdetectionyoloannotated_path

'/root/.cache/kagglehub/datasets/ahemateja19bec1025/trafficsignlocalizationdetectionyoloannotated/versions/9'

In [47]:
!!pip install keras-cv

['Requirement already satisfied: keras-cv in /usr/local/lib/python3.11/dist-packages (0.9.0)',
 'Requirement already satisfied: packaging in /usr/local/lib/python3.11/dist-packages (from keras-cv) (24.2)',
 'Requirement already satisfied: absl-py in /usr/local/lib/python3.11/dist-packages (from keras-cv) (1.4.0)',
 'Requirement already satisfied: regex in /usr/local/lib/python3.11/dist-packages (from keras-cv) (2024.11.6)',
 'Requirement already satisfied: tensorflow-datasets in /usr/local/lib/python3.11/dist-packages (from keras-cv) (4.9.7)',
 'Requirement already satisfied: keras-core in /usr/local/lib/python3.11/dist-packages (from keras-cv) (0.1.7)',
 'Requirement already satisfied: kagglehub in /usr/local/lib/python3.11/dist-packages (from keras-cv) (0.3.10)',
 'Requirement already satisfied: pyyaml in /usr/local/lib/python3.11/dist-packages (from kagglehub->keras-cv) (6.0.2)',
 'Requirement already satisfied: requests in /usr/local/lib/python3.11/dist-packages (from kagglehub->ke

In [48]:
import keras_cv
import keras_core as keras
import tensorflow as tf
from tensorflow.keras import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import *
from keras_cv.models import YOLOV8Backbone, YOLOV8Detector
import os
from keras_cv.losses import YOLOV8Loss
import numpy as np


ImportError: cannot import name 'YOLOV8Loss' from 'keras_cv.losses' (/usr/local/lib/python3.11/dist-packages/keras_cv/api/losses/__init__.py)

In [ ]:
print(tf.__version__)  # Kiểm tra phiên bản TensorFlow

# 1. Preparing

In [ ]:
batch_size=8
epochs=3

In [ ]:
def parse_label_file(label_file_path, max_objects=10):
    with open(label_file_path, 'r') as f:
        lines = f.readlines()
        classes = []
        boxes = []
        for line in lines[:max_objects]:  # Giới hạn số lượng objects
            parts = line.strip().split()
            if len(parts) != 5:
                continue  # Bỏ qua dòng không hợp lệ

            class_id = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])

            classes.append(class_id)
            boxes.append([x_center, y_center, width, height])

        # Padding
        while len(classes) < max_objects:
            classes.append(0)
            boxes.append([0.0, 0.0, 0.0, 0.0])

        return (
            np.array(classes, dtype=np.int32),  # Shape (max_objects,)
            np.array(boxes, dtype=np.float32)    # Shape (max_objects, 4)
        )

def load_image_and_labels(image_path, label_path, max_objects=10):
    # Đọc ảnh
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [640, 640])
    image = image / 255.0
    image = tf.ensure_shape(image, [640, 640, 3])

    # Đọc nhãn
    def parse_label(path):
        path_str = path.numpy().decode('utf-8')
        classes, boxes = parse_label_file(path_str, max_objects)
        return classes, boxes

    classes, boxes = tf.py_function(
        func=parse_label,
        inp=[label_path],
        Tout=[tf.int32, tf.float32]
    )

    # Cố định shape
    classes = tf.ensure_shape(classes, [max_objects])  # Shape (max_objects,)
    boxes = tf.ensure_shape(boxes, [max_objects, 4])   # Shape (max_objects, 4)

    # Tạo dictionary labels
    labels = {
        "classes": classes,
        "boxes": boxes
    }

    return image, labels  # CHỈ MỘT RETURN DUY NHẤT

def create_dataset(image_dir, label_dir, batch_size=8, max_objects=10):
    # Lấy danh sách file ảnh và nhãn
    image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.jpg')]
    label_files = [os.path.join(label_dir, f.replace('.jpg', '.txt')) for f in image_files]

    # Lọc file nhãn tồn tại
    image_files = [img for img, lbl in zip(image_files, label_files) if os.path.exists(lbl)]
    label_files = [lbl for lbl in label_files if os.path.exists(lbl)]

    # Tạo dataset từ cặp (image_path, label_path) với kiểu dữ liệu string
    dataset = tf.data.Dataset.from_tensor_slices(
        (tf.constant(image_files, dtype=tf.string),  # Đảm bảo kiểu string
         tf.constant(label_files, dtype=tf.string))  # Đảm bảo kiểu string
    )

    # Ánh xạ hàm load_image_and_labels
    dataset = dataset.map(
        lambda img_path, lbl_path: load_image_and_labels(img_path, lbl_path, max_objects),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    # Batch và prefetch
    dataset = dataset.batch(batch_size,drop_remainder=True)
    dataset = dataset.prefetch(1)

    return dataset

In [ ]:
# train_image_dir = '/kaggle/input/trafficsignlocalizationdetectionyoloannotated/TrafficSignLocalizationandDetection/train/images'
# train_label_dir = '/kaggle/input/trafficsignlocalizationdetectionyoloannotated/TrafficSignLocalizationandDetection/train/labels'
# val_image_dir = '/kaggle/input/trafficsignlocalizationdetectionyoloannotated/TrafficSignLocalizationandDetection/valid/images'
# val_label_dir = '/kaggle/input/trafficsignlocalizationdetectionyoloannotated/TrafficSignLocalizationandDetection/valid/labels'
# test_image_dir = '/kaggle/input/trafficsignlocalizationdetectionyoloannotated/TrafficSignLocalizationandDetection/test/images'
# test_label_dir = '/kaggle/input/trafficsignlocalizationdetectionyoloannotated/TrafficSignLocalizationandDetection/test/labels'

train_image_dir = f'{ahemateja19bec1025_trafficsignlocalizationdetectionyoloannotated_path}/TrafficSignLocalizationandDetection/train/images'
train_label_dir = f'{ahemateja19bec1025_trafficsignlocalizationdetectionyoloannotated_path}/TrafficSignLocalizationandDetection/train/labels'
val_image_dir = f'{ahemateja19bec1025_trafficsignlocalizationdetectionyoloannotated_path}/TrafficSignLocalizationandDetection/valid/images'
val_label_dir = f'{ahemateja19bec1025_trafficsignlocalizationdetectionyoloannotated_path}/TrafficSignLocalizationandDetection/valid/labels'
test_image_dir = f'{ahemateja19bec1025_trafficsignlocalizationdetectionyoloannotated_path}/TrafficSignLocalizationandDetection/test/images'
test_label_dir = f'{ahemateja19bec1025_trafficsignlocalizationdetectionyoloannotated_path}/TrafficSignLocalizationandDetection/test/labels'

In [ ]:
num_classes=36

In [ ]:
train_dataset = create_dataset(train_image_dir, train_label_dir, batch_size=batch_size)
val_dataset = create_dataset(val_image_dir, val_label_dir, batch_size=batch_size)
test_dataset = create_dataset(test_image_dir, test_label_dir, batch_size=batch_size)

# Kiểm tra shape
for batch in train_dataset.take(1):
    images = batch[0]
    labels = batch[1]
    print("Images shape:", images.shape)  # (8, 640, 640, 3)
    print("Classes shape:", labels["classes"].shape)  # (8, 10)
    print("Boxes shape:", labels["boxes"].shape)  # (8, 10, 4)

In [ ]:
for images, labels in train_dataset.take(1):
    print("Dữ liệu từ train_dataset:")
    print("Shape ảnh:", images.shape)  # (8, 640, 640, 3)
    print("Shape classes:", labels["classes"].shape)  # (8, 10)
    print("Shape boxes:", labels["boxes"].shape)      # (8, 10, 4)
    break

for images, labels in val_dataset.take(1):
    print("\nDữ liệu từ val_dataset:")
    print("Shape ảnh:", images.shape)  # (8, 640, 640, 3)
    print("Shape classes:", labels["classes"].shape)  # (8, 10)
    print("Shape boxes:", labels["boxes"].shape)      # (8, 10, 4)
    break

# 2. Model

In [ ]:
def create_model(pretrained=True, num_classes=80, input_shape=(batch_size, 224, 224, 3)):
    # Tạo input
    input_data = tf.ones(shape=input_shape)

    if pretrained:
        # Sử dụng backbone pretrained
        backbone = YOLOV8Backbone.from_preset("yolo_v8_l_backbone_coco")
        fpn_depth = 2  # Độ sâu FPN phù hợp với preset 'l'
    else:
        # Backbone tự khởi tạo với cấu hình tuỳ chỉnh
        backbone = YOLOV8Backbone(
            stackwise_channels=[128, 256, 512, 1024],
            stackwise_depth=[3, 9, 9, 3],
            include_rescaling=False,
        )
        fpn_depth = 1  # Điều chỉnh theo kiến trúc

    # Tạo detection model với backbone
    detector = YOLOV8Detector(
        num_classes=num_classes,
        bounding_box_format="xywh",
        backbone=backbone,
        fpn_depth=fpn_depth
    )

    # Forward pass
    output = detector(input_data)

    return detector

In [ ]:
def create_light_model(
    pretrained=True,
    num_classes=36,
    input_shape=(640, 640, 3)  # Sửa thành kích thước chuẩn (bội số của 32)
):
    # Khởi tạo backbone
    if pretrained:
        backbone = YOLOV8Backbone.from_preset("yolo_v8_s_backbone_coco")
        fpn_depth = 1
    else:
        backbone = YOLOV8Backbone(
            stackwise_channels=[64, 128, 256, 512],
            stackwise_depth=[2, 4, 4, 2],
            include_rescaling=False,
        )
        fpn_depth = 1

    # Tạo YOLOv8Detector với input_shape chính xác
    detector = YOLOV8Detector(
        num_classes=num_classes,
        bounding_box_format="xywh",
        backbone=backbone,
        fpn_depth=fpn_depth,
        input_shape=input_shape  # Truyền input_shape trực tiếp vào detector
    )

    # Build model để khởi tạo trọng số
    detector.build(input_shape=(None, *input_shape))  # Batch size là None

    return detector

In [ ]:

def create_ligh_model(num_classes=36):
    inputs = Input(shape=(640, 640, 3), name="input_image")
    backbone = YOLOV8Backbone.from_preset("yolo_v8_s_backbone_coco")
    detector = YOLOV8Detector(
        num_classes=num_classes,
        bounding_box_format="xywh",
        backbone=backbone,
        fpn_depth=1
    )
    outputs = detector(inputs)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # Compile with YOLOv8-specific loss
    model.compile(
        optimizer="adam",
        loss=YOLOV8Loss(
            classification_loss="focal",  # Focal loss for class predictions
            box_loss="ciou",              # CIoU loss for boxes
            classification_weight=1.0,    # Adjust weights as needed
            box_weight=1.0
        ),
    )
    return model


In [ ]:

model=create_ligh_model()
model.build(input_shape=(None, 640, 640, 3))  # Batch size = None (linh hoạt)
model.summary()

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model=model, show_shapes= True, dpi=76)

In [ ]:
def define_hyperparameters(learning_rate=2e-5, decay_rate=0.99):
    initial_learning_rate = learning_rate

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=100000,
        decay_rate=0.96,
        staircase=True
    )

    # Use the learning rate schedule in the optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.9, beta_2=0.999)
    metrics = [tf.keras.metrics.Precision(),
              tf.keras.metrics.Recall(),
              tf.keras.metrics.IoU(num_classes=num_classes, target_class_ids=[1])
             ]
    return optimizer, metrics

In [ ]:
optimizer, metrics =define_hyperparameters()

In [ ]:
for images, labels in train_dataset.take(1):
    print("Images shape:", images.shape)  # (8, 640, 640, 3)
    print("Classes shape:", labels["classes"].shape)  # (8, 10, 1)
    print("Boxes shape:", labels["boxes"].shape)  # (8, 10, 4)

In [ ]:
# from keras_cv.losses import CIoULoss

# model.compile(
#     optimizer="adam",
#     loss={
#         'boxes': CIoULoss(bounding_box_format="xywh"),  # Replace 'boxes' with actual output name
#         'classes': 'categorical_crossentropy'           # Replace 'classes' with actual output name
#     },
#     # metrics=... (optional)
# )

In [ ]:
tf.config.optimizer.set_jit(False)  # Tắt XLA

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
)